# KSB Checker

## Set up

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from tkinter import Tk, filedialog

from l3_digital_support_technician import criteria

In [2]:
# Use TkInter to open a file dialog
# Hide the root Tkinter window
root = Tk()
root.withdraw()

# Bring the root window to the front (even though it's hidden)
root.attributes('-topmost', True)
root.lift()
root.focus_force()

In [3]:
# model = OllamaLLM(model="llama3.2")  # Quicker for testing
model = OllamaLLM(model="gpt-oss")  # Better for evaluation


In [4]:
template = """
An apprentice has submitted the following text as part of their portfolio. Note that
images were included in the portfolio, but they are not included in the text below.
Here is the portfolio text:

{portfolio_text}

Your task:
Please now determine if any of the following Knowledge, Skills, and Behaviours (KSBs) have
been met, including whether the apprentice has fulfulled the pass or distinction criteria.
I would like this in the format of a table showing, for each KSB:
- highlights of any evidence from the portfolio,
- whether the pass criteria have been met,
- whether the distinction criteria have been met (if applicable), and
- suggestions for how the evidence could be strengthend.

Please use gender neutral pronouns like 'they' and 'them' if referring to the apprentice. Some
external examiners are quite hard to please. So please only indicate that criteria have been
met when there's little doubt that it's true.

KSBs:
{ksbs}
"""

## Portfolio

In [5]:
# Get the portfolio text file
# This can be created using the extract-text.ipynb notebook
portfolio_filename = filedialog.askopenfilename(
    title="Select TXT files",
    filetypes=[("Text Documents", "*.txt")]
)

portfolio_filename

'C:/Users/jmmck/Documents/Projects/KSB-checker/students/EI/portfolio.txt'

In [6]:
# Open a file dialog for selecting a markdown file to save as
evaluation_filename = filedialog.asksaveasfile(
    title="Select MD files",
    filetypes=[("Markdown Documents", "*.MD")]
)

In [7]:
with open(portfolio_filename, "r", encoding="utf-8") as f:
    portfolio_text = f.read()

## Evaluation

In [8]:
prompt = ChatPromptTemplate.from_template(template)

In [9]:
chain = prompt | model

In [10]:
evaluation_text = ""

In [11]:
for heading, criteria_text in criteria.items():
    print(f"Evaluating: {heading}")
    response = chain.invoke(
        {
            "portfolio_text": f"{portfolio_text}",
            "ksbs": f"{criteria_text}",
        }
    )
    evaluation_text += f"## {heading}\n\n{response}\n\n"


Evaluating: 1. Digital Technologies K1 S1 B2 B4
Evaluating: 2. Digital and Information Security K3 K4 K8 S3
Evaluating: 3. Digital Information Management Systems K7 S5
Evaluating: 4. Communication K10 K11 S6
Evaluating: 5. Digital Learning K12 K15 S7
Evaluating: 6. Continuous Improvement K14 S9 B5
Evaluating: 7. Teamwork K13 S10
Evaluating: 8. Application Skills Support K19 K21 S14


In [12]:
if evaluation_filename:
    with open(evaluation_filename.name, 'w', encoding='utf-8') as f:
        f.write(evaluation_text)